In [7]:
# Import the required Libraries

from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
import pickle, os.path, base64, email 
from bs4 import BeautifulSoup 
import pandas as pd
from fuzzywuzzy import fuzz #pip install fuzzywuzzy

In [2]:

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly'] 
  
# Store the user access token 
# If no valid token found, we will create one. 
creds = None

# The file token.pickle contains the user access token. 
# Check if it exists 
if os.path.exists('../token.pickle'): 

    # Read the token and store it in creds 
    with open('../token.pickle', 'rb') as token: 
        creds = pickle.load(token) 

# If credentials are not available or are invalid, ask the user to log in. 
if not creds or not creds.valid: 
    if creds and creds.expired and creds.refresh_token: 
        creds.refresh(Request()) 
    else: 
        flow = InstalledAppFlow.from_client_secrets_file('../credentials.json', SCOPES) 
        creds = flow.run_local_server(port=0) 

    # Save the access token in token.pickle file for the next run 
    with open('../token.pickle', 'wb') as token: 
        pickle.dump(creds, token) 

# Connect to the Gmail API 
service = build('gmail', 'v1', credentials=creds) 

# request a list of all the messages 
result = service.users().messages().list(userId='me').execute() 

# We can also pass maxResults to get any number of emails. 
messages = result.get('messages') 

inbox = {
    'Subject': [], 'From': [], 'Message':[]
}

# iterate through all the messages 
for msg in messages: 
    # Get the message from its id 
    txt = service.users().messages().get(userId='me', id=msg['id']).execute() 

    # Use try-except to avoid any Errors 
    try: 
        # Get value of 'payload' from dictionary 'txt' 
        payload = txt['payload'] 
        headers = payload['headers'] 

        # Look for Subject and Sender Email in the headers 
        for d in headers: 
            if d['name'] == 'Subject': 
                subject = d['value'] 
            if d['name'] == 'From': 
                sender = d['value'] 

        # The Body of the message is in Encrypted format. So, we have to decode it. 
        parts = payload.get('parts')[0] 
        data = parts['body']['data'] 
        data = data.replace("-","+").replace("_","/") 
        decoded_data = base64.b64decode(data) 

        # Data obtained is in lxml. So, we will parse  
        # it with BeautifulSoup library 
        soup = BeautifulSoup(decoded_data , "lxml") 
        body = soup.body() 

        # Storing the subject, sender's email and message 
        inbox['Subject'].append(subject)
        inbox['From'].append(sender)
        inbox['Message'].append(body)
    except: 
        pass

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=621011074468-ciou7u6tve0s33dtk0dal5sieo9htc6e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57215%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=yIwzVwfXEXKYe8zGpHJM4TDm9mwkr5&access_type=offline


In [6]:
inbox = pd.DataFrame(inbox)
inbox.tail()

,Subject,From,Message
75,I got placed last week in the campus interview...,Quora Digest <digest-noreply@quora.com>,[[Parisha's Digest\r\nYour Quora Digest\r\n\r\...
76,The Perfect Start to Your Data Science Career ...,Mayank <connect@analyticsvidhya.com>,"[[[ ]( # ) \r\n\r\n \r\n[ ]( # ) \r\n""..."
77,"Parisha, you have 68 friend requests",Facebook <notification@facebookmail.com>,[[========================================\r\n...
78,As a fresher how can I introduce myself for a ...,Quora Digest <digest-noreply@quora.com>,[[Parisha's Digest\r\nYour Quora Digest\r\n\r\...
79,50% off for you & a friend,Coursera <no-reply@t.mail.coursera.org>,"[[[ Refer-a-friend... - Coursera - ]\r\n,..."


In [4]:
QUERY_TO_MATCH = 'Thank you for applying'

inbox[inbox.Subject.apply(lambda x: fuzz.token_set_ratio(x.lower(), QUERY_TO_MATCH.lower())) > 80]

,Subject,From,Message
1,Thankyou for applying,Sameeksha Vishnoi <sameekshavishnoi3@gmail.com>,[[Hi\r\n]]
2,Thanks for applying,Sameeksha Vishnoi <sameekshavishnoi3@gmail.com>,[[Hey\r\n]]
3,Thank u for applying,Twinkle Rastogi <twinklerastogi1998@gmail.com>,[[Y\r\n]]
4,Thanks for applying,Twinkle Rastogi <twinklerastogi1998@gmail.com>,[[Hi\r\n]]
